In [1]:
import numpy as np
import copy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from sumo_interface import start_sim

In [2]:
def random_dur(default_dur, sigma=1):
	default_dur = sum(default_dur,[])
	result =  np.clip(np.random.normal(default_dur,sigma*np.ones(len(default_dur)),len(default_dur)),0.5,60)
	#print("generated: ",result)
	return result

In [3]:
def evaluation(file_path,id_TLs,dur_TLs_):
	dur_TLs = copy.deepcopy(dur_TLs_)
	#print("eval ind: ",dur_TLs)
	edit_net(file_path,id_TLs,dur_TLs)
	#--> to sumo api

	start_sim(file_path+"/osm.sumocfg")
	import xml.etree.ElementTree as ET
	from numpy import mean
	tree = ET.ElementTree(file = "/home/lin/workspace/GA/GA_Traffic_Optimization/result.xml")
	#tree = ET.ElementTree(file = "/home/zhenyuli/workspace/GA_Traffic_Optimization/result.xml")
	trip_infos = tree.getroot()
	timeLoss = mean([float(trip.attrib['timeLoss']) for trip in trip_infos.findall("tripinfo")])

	print("##############################################################")
	print("##############################################################")
	print("avg time loss: ", timeLoss)
	print("##############################################################")
	print("##############################################################")
	return timeLoss,

In [4]:
def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrapper
    return decorator

In [5]:
def ga(file_path,default_dur,id_TLs):
	#parameters
	INIT_SIZE = 100
	MAX_ITER = 30
	SIGMA = 1
	P_CROSSOVER = 0.5
	P_MUTATION = 0.1
	MIN = 0.5
	MAX = 60

	#DEAP
	creator.create("FitnessMin",base.Fitness,weights=(-1.0,)) 
	creator.create("Individual",list,fitness=creator.FitnessMin) #individuals are returned in list

	toolbox = base.Toolbox()
	toolbox.register("attr_item", random_dur, default_dur, SIGMA)
	toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_item)
	toolbox.register("population", tools.initRepeat, list, toolbox.individual)

	toolbox.register("evaluate", evaluation, file_path,id_TLs)
	toolbox.register("edit_net", edit_net,file_path,id_TLs,dur_TLs)
	toolbox.register("start_sim",start_sim,file_path)
	toolbox.register("mate", tools.cxOnePoint)
	#toolbox.register("mate", crossover)23.789012557077626
	toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=SIGMA, indpb=0.1)
	toolbox.decorate("mutate", checkBounds(MIN, MAX))# Bounds are still needed to be set.

	toolbox.register("select", tools.selTournament, tournsize=5)
	#toolbox.decorate("select", printsel())

	pop = toolbox.population(n=INIT_SIZE)

	hof = tools.HallOfFame(1) #pick the best one
	stats = tools.Statistics(lambda ind: ind.fitness.values)
	stats.register("avg", np.mean)
	stats.register("std", np.std)
	stats.register("min", np.min)
	stats.register("max", np.max)

	pop,log = algorithms.eaSimple(pop,toolbox,cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_ITER,stats=stats,halloffame=hof,verbose=True)

	return hof,log

In [6]:
def get_default_duration(file_path):
	import xml.etree.ElementTree as ET

	tree = ET.ElementTree(file = file_path+"/osm.net.xml")
	net = tree.getroot()
	trafficLights = net.findall("tlLogic")

	id_TLs = []
	dur_TLs = []
	for tl in trafficLights:
		id_TLs.append(tl.attrib['id'])
		durations = [phase.attrib['duration'] for phase in tl]
		dur_TLs.append(durations)

	return id_TLs,dur_TLs

In [7]:
def edit_net(file_path,id_TLs,dur_TLs):
	import xml.etree.ElementTree as ET

	tree = ET.ElementTree(file = file_path+"/osm.net.xml")
	net = tree.getroot()
	trafficLights = net.findall("tlLogic")

	tl_idx = [[tl.attrib['id'] for tl in trafficLights].index(id) for id in id_TLs]

	for i in tl_idx:
		nb_phase = len(trafficLights[i])
		for j in range(nb_phase):
			trafficLights[i][j].attrib["duration"] = str(dur_TLs.pop(0))
		#for j in range(len(dur_TLs[i])):
		#	trafficLights[i][j].attrib["duration"] = dur_TLs[i][j]

	tree.write(file_path+"/osm.net.xml")

In [ ]:
MAP_PATH = "/home/lin/workspace/GA/GA_Traffic_Optimization/wuhan"
#MAP_PATH = "/home/zhenyuli/Sumo/2020-02-08-00-46-38"
id_TLs, dur_TLs = get_default_duration(MAP_PATH)

best_solution,log = ga(MAP_PATH,dur_TLs,id_TLs)
print("best solution is: ",best_solution[0])
print("best time loss", best_solution[0].fitness.values)

 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  138.49571062375392
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  270.7303674166904
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  169.41381657647395
##############################################################
##############################################################
 Retrying in 1 seconds
################################################

##############################################################
##############################################################
avg time loss:  138.18058387923668
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  174.38579891768728
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  196.4353346624893
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
########

##############################################################
##############################################################
avg time loss:  128.53364568499003
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  141.92954144118485
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  135.73894047279975
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#######

##############################################################
##############################################################
avg time loss:  182.52863856451154
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  191.17518370834523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  181.023716889775
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#########

##############################################################
##############################################################
avg time loss:  159.81950726288807
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  180.30442039305044
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  197.5758957561948
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
########

##############################################################
##############################################################
avg time loss:  211.76531187695812
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  166.2836627741384
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  242.82429792082024
##############################################################
##############################################################
 Retrying in 1 seconds


In [ ]:
gen = log.select("gen")
fit_mins = log.select("min")
import plotly.graph_objects as go
import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=gen, y=fit_mins, mode='lines+markers'))
fig.update_layout(title='The performance of GA')
fig.show()

In [ ]:
print(log)